In [1]:
import numpy as np
import pandas as pd

In [2]:
# load full data.
data_full = pd.read_csv("merged-multiples-fix-ID-age-sex-fix.csv")
data_full_index_list = data_full.unique_id.to_list()
data_full = data_full.set_index('unique_id')

# load analyzed data.
data_analyzed = pd.read_csv("03162022-metadata-analyzed-ID.csv")
data_analyzed_ID_list = data_analyzed.patient_id.to_list()
data_analyzed = data_analyzed.set_index('unique_id')


In [3]:
# check which patient ID already anlyzed, if anlyzed, remove it from full data.
for ID_analyzed in data_analyzed_ID_list:
    data_full = data_full.drop(data_full.index[data_full['patient_id']==ID_analyzed])

# export the dataframe which are patient IDs not analyzed yet
# data_full is not the original full data, its the updated data which only contain not analyzed patient IDs.
data_full.to_csv("not_analyzed_patient_id.csv")

In [4]:
# get the unique patient ID after remove analyzed data
# check how many patient IDs not analyzed yet
patient_id_list = data_full.patient_id.to_list()
patient_id_list_no_dup = list(set(patient_id_list))
print("there are " + len(patient_id_list_no_dup) + " patients not analyzed yet")
print(patient_id_list_no_dup)

[10241, 10246, 10247, 8205, 10257, 10258, 10267, 10268, 10269, 10271, 8224, 10272, 10275, 10280, 10281, 10283, 10285, 8239, 10294, 10296, 8252, 10301, 10302, 8258, 10306, 10310, 10312, 10321, 10333, 8285, 8287, 8288, 8294, 10347, 10352, 10354, 10356, 10361, 10364, 10365, 10367, 10369, 10371, 10376, 8337, 10390, 10391, 10394, 10399, 10400, 8354, 10406, 8363, 10412, 8366, 8367, 10414, 8370, 10419, 10422, 10425, 10428, 10429, 10430, 10431, 8384, 10432, 8386, 10433, 10436, 8385, 10439, 10441, 10443, 10449, 10451, 10453, 10454, 10455, 10456, 8409, 10457, 10459, 10458, 10461, 8416, 10467, 8425, 10474, 10479, 10480, 8437, 10488, 8445, 8454, 8457, 10511, 8464, 10513, 10520, 8475, 10529, 8492, 8494, 10542, 8499, 8502, 10550, 8516, 10565, 8520, 8522, 10570, 10584, 8547, 6504, 8554, 6507, 8556, 6512, 8588, 8593, 8611, 8619, 8642, 8647, 8687, 8691, 8702, 8704, 8708, 8709, 8712, 8713, 8717, 8719, 8724, 8739, 8752, 10803, 8765, 8771, 8787, 10835, 8796, 8797, 10846, 10848, 8804, 10854, 8833, 8844, 88

In [5]:
# create lists, which later can be combined into dataframe.
non_progressed_patient_id = []
non_progressed_age = []
non_progressed_sex = []
non_progressed_stage = []

progressed_patient_id = []
progressed_age = []
progressed_sex = []
progressed_stage = []

for patient_id in patient_id_list_no_dup:
    # subset the full data into different patient IDs dataframe.
    subset_ID_df = data_full[data_full["patient_id"].isin([patient_id])]
    print(subset_ID_df)
    # transfer each column in subset dataframe into lists.
    stage_list = subset_ID_df.stage.to_list()
    age_list = subset_ID_df.visit_age.to_list()
    sex_list = subset_ID_df.sex_lett.to_list()
    stage_list = subset_ID_df.stage.to_list()
    # combine column years and stage into dictionary so that we can compare
    # the stage (value) in one year (key) to the stage in another year.
    subset_ID_dict = dict(zip(subset_ID_df.visit_yr, subset_ID_df.stage))
    print(subset_ID_dict)
    # we dont care about all A, all B, all C, all D.
    if len(list(set(stage_list))) > 1:
        # remove duplicates but keep list order!!! Careful.
        # check if the stages only contain C, D, it it is, then remove it since C/D is not applicable.
        if sorted(set(stage_list)) == ['C', 'D']:
            print('patient ID ' + str(patient_id) + ' progressed but only C and D, not applicable')
        # then check years of stage. The data is ordered by stage from A to D. So in subset dataframe,
        # so if the last one in years list is the oldest year (largest), it mean the progression is follow the years.
        # for example, dictionary: {2017:A, 2018:B, 2019:C}, the progression is follow the years.
        # for example, dictionary: {2017:A, 2019:B, 2018:C}, the progression is not follow the years so it mean recovered. 
        elif max(subset_ID_dict.keys()) == list(subset_ID_dict.keys())[len(subset_ID_dict.keys()) - 1]:
            print('patient ID ' + str(patient_id) + ' progressed')
            # put those progressed patient ID, age, sex info into corresponding lists.
            progressed_patient_id.append(patient_id)
            progressed_age.append(age_list)
            progressed_sex.append(sex_list)
            progressed_stage.append(stage_list)
        else: # if the last one is not the oldest year, it mean the stage progressed but 
              # recovered back so the last year in dictionary is not the oldesst.
            print('patient ID ' + str(patient_id) + ' progressed but recovered')
    else:# for the rest of patient IDs, they are not progressed
        print('patient ID ' + str(patient_id) + ' didnt progress')
        # put those not progress patient ID, age, sex info into corresponding lists.
        non_progressed_patient_id.append(patient_id)
        non_progressed_age.append(age_list)
        non_progressed_sex.append(sex_list)
        non_progressed_stage.append(stage_list)

           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
1008            10241      2016     A        33   Female
1009            10241      2018     A   unknown  unknown
{2016: 'A', 2018: 'A'}
patient ID 10241 didnt progress
           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
1010            10246      2016     A        57   Female
1011            10246      2018     A   unknown  unknown
{2016: 'A', 2018: 'A'}
patient ID 10246 didnt progress
           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
1012            10247      2016     A        40   Female
1013            10247      2017     A   unknown  unknown
{2016: 'A', 2017: 'A'}
patient ID 10247 didnt progress
           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
480              8205      2017     A

           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
568              8520      2017     A   unknown  unknown
569              8520      2018     B   unknown  unknown
{2017: 'A', 2018: 'B'}
patient ID 8520 progressed
           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
570              8522      2017     A   unknown  unknown
571              8522      2018     A   unknown  unknown
{2017: 'A', 2018: 'A'}
patient ID 8522 didnt progress
           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
1181            10570      2017     A        92   Female
1182            10570      2020     C   unknown  unknown
{2017: 'A', 2020: 'C'}
patient ID 10570 progressed
           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
1183            10584      2018     A   unknown

           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
141              7192      2015     A        54     Male
142              7192      2017     A   unknown  unknown
143              7192      2018     A   unknown  unknown
{2015: 'A', 2017: 'A', 2018: 'A'}
patient ID 7192 didnt progress
           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
726              9241      2016     A        57   Female
727              9241      2017     A   unknown  unknown
{2016: 'A', 2017: 'A'}
patient ID 9241 didnt progress
           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
146              7194      2022     A        62  unknown
{2022: 'A'}
patient ID 7194 didnt progress
           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
147              7209      2015     A   

338              7592      2022     A   unknown  unknown
{2018: 'A', 2022: 'A'}
patient ID 7592 didnt progress
           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
339              7594      2017     A   unknown  unknown
340              7594      2022     A   unknown  unknown
{2017: 'A', 2022: 'A'}
patient ID 7594 didnt progress
           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
830              9642      2016     A        56   Female
831              9642      2017     A   unknown  unknown
{2016: 'A', 2017: 'A'}
patient ID 9642 didnt progress
           patient_id  visit_yr stage visit_age sex_lett
unique_id                                               
341              7595      2017     B   unknown  unknown
342              7595      2018     C   unknown  unknown
{2017: 'B', 2018: 'C'}
patient ID 7595 progressed
           patient_id  visit_yr stage visit_age

In [6]:
# put corresponding lists into dataframe by zip function.
non_progressed_df = pd.DataFrame(list(zip(non_progressed_patient_id, non_progressed_age, non_progressed_sex, non_progressed_stage)), columns=['patient_id', 'age', 'sex', 'stage'])
progressed_df = pd.DataFrame(list(zip(progressed_patient_id, progressed_age, progressed_sex, progressed_stage)), columns=['patient_id', 'age', 'sex', 'stage'])

In [7]:
# export the dataframe into csv.
non_progressed_df.to_csv('non_progressed_data.csv')
progressed_df.to_csv('progressed_data.csv')